# Tipología de Democracias en el Mundo (2025) – Análisis de Clustering



## Introducción

La democracia adopta múltiples formas y niveles de calidad.  
Este proyecto analiza **cómo se agrupan los países del mundo** según sus características democráticas, utilizando indicadores desarrollados por el prestigioso proyecto **V-Dem (Varieties of Democracy)**.

V-Dem es una iniciativa internacional coordinada por la **Universidad de Gotemburgo** (Suecia) y la **Universidad de Notre Dame** (Estados Unidos), que trabaja junto con una red global de más de **3.500 expertos** para evaluar la calidad institucional y democrática de países en todo el mundo.

Los datos de V-Dem constituyen **el estándar más completo** en estudios comparados de democracia, midiendo dimensiones como:

- libertades civiles  
- participación ciudadana  
- independencia judicial  
- libertad de prensa  
- igualdad política  
- control del poder y rendición de cuentas  

El objetivo de este proyecto es **descubrir tipologías de democracias** en el presente, combinando técnicas de **Machine Learning no supervisado** con interpretación politológica para proponer una clasificación empírica de los regímenes políticos contemporáneos.

**Dataset:** `V-Dem-CY-Core-v15.csv`  
- Versión 15 (marzo 2025)  
- Datos país–año con +1.000 indicadores de democracia, derechos y participación  
- Se utilizará el **año más reciente disponible** para una visión actual del panorama democrático global

---

## Preguntas guía
- ¿Qué tipos de democracia se observan hoy en el mundo?  
- ¿Qué indicadores politológicos definen mejor esas diferencias?  
- ¿Qué países conforman democracias **liberales**, **electorales**, **delegativas** o **autoritarismos**?

---

## Objetivos
- Seleccionar indicadores centrales sobre calidad democrática (instituciones, libertades, participación).  
- Aplicar **PCA** para reducción de dimensionalidad.  
- Entrenar **K-Means** (y/o DBSCAN) para identificar clusters.  
- Visualizar los agrupamientos en 2D y **analizar sus implicancias politológicas**.

---

## Alcance y supuestos
- Se utiliza información pública y consolidada de V-Dem.  
- Se trabaja a nivel **comparado** (todos los países con datos disponibles).  
- El análisis se limita a variables seleccionadas y al año más reciente con cobertura global.  
- El resultado **no implica juicios normativos**, sino una clasificación basada en datos.  

Si bien la medición de la democracia es un campo atravesado por debates conceptuales y disputas normativas, este proyecto no busca cuestionar ni reformular las definiciones adoptadas por V-Dem. El foco se centra en **aplicar técnicas de análisis no supervisado** para identificar patrones y agrupamientos **en función de los datos disponibles**, reconociendo que toda clasificación politológica es una representación parcial de fenómenos complejos.

---

## Reproducibilidad
- Dependencias detalladas en `requirements.txt`  
- Estructura del proyecto basada en mejores prácticas de Data Science  
- Notebook y scripts alojados en `/notebooks` y `/src`  

**Citación recomendada:**  
V-Dem Institute (2025). *V-Dem Country-Year Dataset v15*. Gothenburg: University of Gothenburg and University of Notre Dame.  
Disponible en: https://www.v-dem.net/data/the-v-dem-dataset/

## 0. Configuración inicial

En esta sección se importan las librerías necesarias y se definen configuraciones visuales globales  
que se aplicarán a lo largo del notebook.

In [2]:
# --- Importaciones principales ---
import pandas as pd
import numpy as np

# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Preprocesamiento ---
from sklearn.preprocessing import StandardScaler

# --- Reducción de dimensionalidad ---
from sklearn.decomposition import PCA

# --- Clustering ---
from sklearn.cluster import KMeans

# --- Evaluación del clustering ---
from sklearn.metrics import silhouette_score

In [3]:
# --- Configuración visual ---

# Estilo general de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette('Set2')

# Mostrar todas las columnas al inspeccionar DataFrames
pd.set_option('display.max_columns', None)

## 1. Carga e inspección inicial de los datos

En esta sección se importa el dataset y se realiza una primera exploración general.  
El objetivo es verificar la estructura, tipos de datos, valores nulos o duplicados,  
y obtener una visión preliminar del contenido antes del análisis detallado.

In [ ]:
# --- Carga de datos ---
file_path = "../data/raw/V-Dem-CY-Core-v15.csv"
df = pd.read_csv(file_path, low_memory=False)

country_name country_text_id  country_id  year historical_date  project  \
0       Mexico             MEX           3  1789      1789-12-31        1   
1       Mexico             MEX           3  1790      1790-12-31        1   
2       Mexico             MEX           3  1791      1791-12-31        1   
3       Mexico             MEX           3  1792      1792-12-31        1   
4       Mexico             MEX           3  1793      1793-12-31        1   

   historical                  histname  codingstart  codingend  \
0           1  Viceroyalty of New Spain         1789       2024   
1           1  Viceroyalty of New Spain         1789       2024   
2           1  Viceroyalty of New Spain         1789       2024   
3           1  Viceroyalty of New Spain         1789       2024   
4           1  Viceroyalty of New Spain         1789       2024   

   codingstart_contemp  codingend_contemp  codingstart_hist  codingend_hist  \
0               1900.0             2024.0            1789.0          1920.0   
1               1900.0             2024.0            1789.0          1920.0   
2               1900.0             2024.0            1789.0          1920.0   
3               1900.0             2024.0            1789.0          1920.0   
4               1900.0             2024.0            1789.0          1920.0   

   gapstart1  gapstart2  gapstart3  gapend1  gapend2  gapend3  gap_index  \
0        NaN        NaN        NaN      NaN      NaN      NaN          1   
1        NaN        NaN        NaN      NaN      NaN      NaN          1   
2        NaN        NaN        NaN      NaN      NaN      NaN          1   
3        NaN        NaN        NaN      NaN      NaN      NaN          1   
4        NaN        NaN        NaN      NaN      NaN      NaN          1   

   COWcode  v2x_polyarchy  v2x_polyarchy_codelow  v2x_polyarchy_codehigh  \
0     70.0          0.028                  0.016                   0.037   
1     70.0          0.028                  0.016                   0.037   
2     70.0          0.028                  0.016                   0.037   
3     70.0          0.028                  0.016                   0.037   
4     70.0          0.028                  0.016                   0.037   

   v2x_polyarchy_sd  v2x_libdem  v2x_libdem_codelow  v2x_libdem_codehigh  \
0             0.011       0.044               0.026                0.055   
1             0.011       0.044               0.026                0.055   
2             0.011       0.044               0.026                0.055   
3             0.011       0.044               0.026                0.055   
4             0.011       0.044               0.026                0.055   

   v2x_libdem_sd  v2x_partipdem  v2x_partipdem_codelow  \
0          0.014          0.006                  0.001   
1          0.014          0.006                  0.001   
2          0.014          0.006                  0.001   
3          0.014          0.006                  0.001   
4          0.014          0.006                  0.001   

   v2x_partipdem_codehigh  v2x_partipdem_sd  v2x_delibdem  \
0                    0.01             0.007           NaN   
1                    0.01             0.007           NaN   
2                    0.01             0.007           NaN   
3                    0.01             0.007           NaN   
4                    0.01             0.007           NaN   

   v2x_delibdem_codelow  v2x_delibdem_codehigh  v2x_delibdem_sd  v2x_egaldem  \
0                   NaN                    NaN              NaN          NaN   
1                   NaN                    NaN              NaN          NaN   
2                   NaN                    NaN              NaN          NaN   
3                   NaN                    NaN              NaN          NaN   
4                   NaN                    NaN              NaN          NaN   

   v2x_egaldem_codelow  v2x_egaldem_codehigh  v2x_egaldem_sd  v2x_api  \
0                  N

In [8]:
# --- Vista general y estructura ---
print("Dimensiones:", df.shape)
display(df.head())
print("\nInformación general del DataFrame:\n")
df.info()

Dimensiones: (27913, 1818)


country_name country_text_id  country_id  year historical_date  project  \
0       Mexico             MEX           3  1789      1789-12-31        1   
1       Mexico             MEX           3  1790      1790-12-31        1   
2       Mexico             MEX           3  1791      1791-12-31        1   
3       Mexico             MEX           3  1792      1792-12-31        1   
4       Mexico             MEX           3  1793      1793-12-31        1   

   historical                  histname  codingstart  codingend  \
0           1  Viceroyalty of New Spain         1789       2024   
1           1  Viceroyalty of New Spain         1789       2024   
2           1  Viceroyalty of New Spain         1789       2024   
3           1  Viceroyalty of New Spain         1789       2024   
4           1  Viceroyalty of New Spain         1789       2024   

   codingstart_contemp  codingend_contemp  codingstart_hist  codingend_hist  \
0               1900.0             2024.0            1789.0          1920.0   
1               1900.0             2024.0            1789.0          1920.0   
2               1900.0             2024.0            1789.0          1920.0   
3               1900.0             2024.0            1789.0          1920.0   
4               1900.0             2024.0            1789.0          1920.0   

   gapstart1  gapstart2  gapstart3  gapend1  gapend2  gapend3  gap_index  \
0        NaN        NaN        NaN      NaN      NaN      NaN          1   
1        NaN        NaN        NaN      NaN      NaN      NaN          1   
2        NaN        NaN        NaN      NaN      NaN      NaN          1   
3        NaN        NaN        NaN      NaN      NaN      NaN          1   
4        NaN        NaN        NaN      NaN      NaN      NaN          1   

   COWcode  v2x_polyarchy  v2x_polyarchy_codelow  v2x_polyarchy_codehigh  \
0     70.0          0.028                  0.016                   0.037   
1     70.0          0.028                  0.016                   0.037   
2     70.0          0.028                  0.016                   0.037   
3     70.0          0.028                  0.016                   0.037   
4     70.0          0.028                  0.016                   0.037   

   v2x_polyarchy_sd  v2x_libdem  v2x_libdem_codelow  v2x_libdem_codehigh  \
0             0.011       0.044               0.026                0.055   
1             0.011       0.044               0.026                0.055   
2             0.011       0.044               0.026                0.055   
3             0.011       0.044               0.026                0.055   
4             0.011       0.044               0.026                0.055   

   v2x_libdem_sd  v2x_partipdem  v2x_partipdem_codelow  \
0          0.014          0.006                  0.001   
1          0.014          0.006                  0.001   
2          0.014          0.006                  0.001   
3          0.014          0.006                  0.001   
4          0.014          0.006                  0.001   

   v2x_partipdem_codehigh  v2x_partipdem_sd  v2x_delibdem  \
0                    0.01             0.007           NaN   
1                    0.01             0.007           NaN   
2                    0.01             0.007           NaN   
3                    0.01             0.007           NaN   
4                    0.01             0.007           NaN   

   v2x_delibdem_codelow  v2x_delibdem_codehigh  v2x_delibdem_sd  v2x_egaldem  \
0                   NaN                    NaN              NaN          NaN   
1                   NaN                    NaN              NaN          NaN   
2                   NaN                    NaN              NaN          NaN   
3                   NaN                    NaN              NaN          NaN   
4                   NaN                    NaN              NaN          NaN   

   v2x_egaldem_codelow  v2x_egaldem_codehigh  v2x_egaldem_sd  v2x_api  \
0                  N


Información general del DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27913 entries, 0 to 27912
Columns: 1818 entries, country_name to v2xca_academ_sd
dtypes: float64(1801), int64(13), object(4)
memory usage: 387.2+ MB


In [9]:
# --- Verificación de años disponibles ---
years = df['year'].unique()
np.sort(years)

array([1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799,
       1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810,
       1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821,
       1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832,
       1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843,
       1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854,
       1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865,
       1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876,
       1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887,
       1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898,
       1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909,
       1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920,
       1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931,
       1932, 1933, 1934, 1935, 1936, 1937, 1938, 19

In [17]:
pd.set_option('display.max_rows', None)
df['year'].value_counts().sort_index(ascending=False)

year
2024    179
2023    179
2022    179
2021    179
2020    179
2019    179
2018    179
2017    179
2016    179
2015    179
2014    179
2013    179
2012    179
2011    179
2010    178
2009    178
2008    178
2007    178
2006    177
2005    177
2004    177
2003    177
2002    177
2001    177
2000    177
1999    177
1998    176
1997    175
1996    175
1995    175
1994    175
1993    175
1992    174
1991    173
1990    172
1989    158
1988    157
1987    157
1986    157
1985    157
1984    157
1983    157
1982    157
1981    157
1980    157
1979    157
1978    157
1977    157
1976    157
1975    158
1974    158
1973    158
1972    158
1971    158
1970    156
1969    156
1968    156
1967    156
1966    156
1965    156
1964    156
1963    156
1962    156
1961    156
1960    155
1959    156
1958    156
1957    156
1956    156
1955    156
1954    156
1953    156
1952    156
1951    156
1950    155
1949    156
1948    155
1947    152
1946    150
1945    149
1944    149
1943    148
1942    148